# Пятое домашнее задание
Полиенко Анастасия Николаевна, НПМбд-01-19 (вторая подгруппа)

Целью данной домашней работы является изучение библиотеки numpy и применение её функций и методов для решения СЛАУ посредством LU- и LUP-разложений.

Для начала необходимо импортировать библиотеку numpy, чтобы можно было её использовать.
Также импортируем библиотеку scipu.linalg для проверки решений СЛАУ.

In [123]:
import numpy as np
import scipy.linalg as sl

Для начала реализуем функцию, которая будет менять местами строки матрицы. Она в дальшейшем будет использоваться в LUP-разложении.

In [124]:
def switch(A,i,j):
    '''
    Меняет местами i-ую и j-ую строки в матрице А
    '''
    temp = np.copy(A[i])
    A[i] = A[j]
    A[j] = temp
    return A

# LU-разложение

$\textbf{LU-разложение}$ разложение матрицы $A$ --- это представление матрицы $A$ в виде произведения $A = L U$, где $L$ --- нижняя треугольная матрица, $U$ --- верхняя треугольная матрица. 

LU-разложение существует только в том случае, когда матрица $A$ обратима, а все ведущие (угловые) миноры матрицы $A$ невырождены.

Для получения LU-разложения необходимо к матрице $A$ приписать справа единичную матрицу $E$. Приводим матрицу $A | E$ к ступенчатому виду. Таким образом, получаем матрицу $U | L_0$, где $U$ --- верхняя треугольная матрица, а $L_0$ --- нижняя треугольная матрица.

Заметим, что матрица $L_0$ приводит $A$ к ступенчатому виду: $L_0 A = U$. Тогда $A = L_0^{-1} U$. 

Обозначим $L = L_0^{-1}$. Таким образом, получаем разложение $A = L U$.

Ниже представлена функция, находящая LU-разложение матрицы $А$.

In [125]:
def LU(A):
    '''
    Находит матрицы L и U для LU-разложения матрицы F
    '''
    N = len(A)
    E = np.identity(N)
    B = np.copy(A)
    #приведение к ступенчатому виду матрицы A|E
    for i in range(N - 1):
        mult = np.array(B[(i + 1):,i] / (- B[i, i]))
        mult = np.array([mult,]).T
        B_add = mult.dot([B[i],])
        E_add = mult.dot([E[i],])
        B[(i + 1):] += B_add
        E[(i + 1):] += E_add
    U = np.copy(B)
    L = np.linalg.inv(E)
    return L, U

Пусть дана СЛАУ вида $A x = b$. С помощью LU-разложения её можно свести к $L U x = b$.

Эта система может быть решена в два шага:

1. Решается система $L y = b$, которая решается прямой подстановкой, т.к. $L$ --- нижняя треугольная матрица.

2. Решается система $U x = y$, которая решается обратной подстановкой, т.к. $U$ --- верхняя треугольная матрица.

Ниже представлена функция, находящая решение СЛАУ при помощи LU-разложения.

In [126]:
def solve_LU(A,b):
    '''
    Находит решение СЛАУ Ax = b при помощи LU-разложения
    '''
    #проверки
    if A.shape[0] != A.shape[1]:
        return 'Матрица не квадратная'
    if np.linalg.det(A) == 0:
        return 'Матрица вырождена'
    N = len(A)
    for i in range(1, N):
        if np.linalg.det(A[:i, :i]) == 0:
            return 'Главный минор вырожден'
    L, U = LU(A)
    #решаем СЛАУ
    y = np.zeros(N)
    x = np.zeros(N)
    #прямая подстановка
    for i in range(N):
        y[i] = b[i] - L[i].dot(y)
    #обратная подстановка
    for i in range(N - 1, -1, -1):
        x[i] = (y[i] - U[i].dot(x)) / U[i, i]
    return x             

# LUP-разложение

$\textbf{LUP-разложение}$ разложение матрицы $A$ --- это представление матрицы $A$ в виде произведения $P A = L U$, где $L$ --- нижняя треугольная матрица, $U$ --- верхняя треугольная матрица, $P$ --- матрица перестановокю

В отличие от LU-разложения, LUP-разложение существует для любой невырожденной матрицы.

Для получения LUP-разложения необходимо к матрице $A$ приписать справа единичную матрицу $E$. Приводим матрицу $A | E$ к ступенчатому виду с выбором наибольшего по модулю ведущего элемента. Таким образом, получаем матрицу $U | L_0$, где $U$ --- верхняя треугольная матрица.

Т.к. при этом процессе могла понадобится перестановка строк, то введём $P A = A_1$, где $A_1$ --- матрица, для которой приведение к ступенчатому виду с выбором максимального по модулю не требует перестановок.

В итоге также получаем матрицы $P$ и $L = P L_0^{-1}$, которые и приводят к искомому разложению $P A = L U$

Ниже представлена функция, находящая LUP-разложение матрицы $А$.

In [127]:
def LUP(A):
    '''
    Находит матрицы L, U и P для LUP-разложения матрицы A
    '''
    N = len(A)
    P = np.identity(N)
    E = np.identity(N)
    B = np.copy(A)
    #приведение к ступенчатому виду матрицы A|E с учетом максимумов
    for i in range(N - 1):
        max_ind = np.argmax(abs(B[:, i]))
        if i != max_ind:
            B = switch(B, i, max_ind)
            E = switch(E, i, max_ind)
            P = switch(P, i, max_ind)
        mult = np.array(B[(i + 1):,i] / (- B[i, i]))
        mult = np.array([mult,]).T
        B_add = mult.dot([B[i],])
        E_add = mult.dot([E[i],])
        B[(i + 1):] += B_add
        E[(i + 1):] += E_add
    U = np.copy(B)
    L = P.dot(np.linalg.inv(E))
    return L, U, P

Пусть дана СЛАУ вида $A x = b$. С помощью LUP-разложения, домножив левую и правую части на $P$, её можно свести к $L U x = P b$.

Эта система может быть решена в два шага:

1. Решается система $L y = P b$, которая решается прямой подстановкой, т.к. $L$ --- нижняя треугольная матрица.

2. Решается система $U x = y$, которая решается обратной подстановкой, т.к. $U$ --- верхняя треугольная матрица.

Ниже представлена функция, находящая решение СЛАУ при помощи LUP-разложения.

In [128]:
def solve_LUP(A, b):
    '''
    Находит решение СЛАУ Ax = b при помощи LUP-разложения
    '''
    if A.shape[0] != A.shape[1]:
        return 'Матрица не квадратная'
    if np.linalg.det(A) == 0:
        return 'Матрица вырождена'
    N = len(A)
    L, U, P = LUP(A)
    #решаем СЛАУ
    b = P.dot(b)
    y = np.zeros(N)
    x = np.zeros(N)
    for i in range(N):
        y[i] = b[i] - L[i].dot(y)
    for i in range(N - 1, -1, -1):
        x[i] = (y[i] - U[i].dot(x)) / U[i, i]
    return x   

# Проверка решений

Проверим, как работают написанные нами функции. Решим систему 
\begin{equation*}
 \begin{cases}
   x_1 + 2 x_2 + x_3 = 0\\
   2 x_1 + x_2 + x_3 = 1\\
   x_1 - x_2 + 2 x_3 = 0
 \end{cases}
\end{equation*}

Её аналитическое решение: 
\begin{equation*}
 \begin{cases}
   x_1 = \frac{5}{6}\\
   x_2 = -\frac{1}{6}\\
   x_3 = -\frac{1}{2}
 \end{cases}
\end{equation*}

In [129]:
matrix = np.array([[1.,2,1],[2,1,1],[1,-1,2]])
vector = np.array([0.,1,0])

In [130]:
L, U = LU(matrix)
x = solve_LU(matrix,vector)
print('L =', L)
print('U = ', U)
for i in range(len(matrix)):
    print(f'x_{i + 1} = {x[i]}')

L = [[ 1. -0. -0.]
 [ 2.  1.  0.]
 [ 1.  1.  1.]]
U =  [[ 1.  2.  1.]
 [ 0. -3. -1.]
 [ 0.  0.  2.]]
x_1 = 0.8333333333333333
x_2 = -0.16666666666666666
x_3 = -0.5


In [131]:
L, U, P = LUP(matrix)
x = solve_LUP(matrix,vector)
print('L =', L)
print('U =', U)
print('P =', P)
for i in range(len(matrix)):
    print(f'x_{i + 1} = {x[i]}')

L = [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -1.   1. ]]
U = [[2.  1.  1. ]
 [0.  1.5 0.5]
 [0.  0.  2. ]]
P = [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
x_1 = 0.8333333333333333
x_2 = -0.16666666666666666
x_3 = -0.5


In [132]:
x = sl.solve(matrix,vector)
P, L, U = sl.lu(matrix)
print('L =', L)
print('U =', U)
print('P =', P.T)
for i in range(len(matrix)):
    print(f'x_{i + 1} = {x[i]}')

L = [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -1.   1. ]]
U = [[2.  1.  1. ]
 [0.  1.5 0.5]
 [0.  0.  2. ]]
P = [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
x_1 = 0.8333333333333334
x_2 = -0.16666666666666666
x_3 = -0.5


Можем увидеть, что написанные функции для решения СЛАУ посредством LU- и LUP-разложений работают верно, т.к. ответы сходятся и с решением, вычисленным через библиотеку scipy, и с аналитическим решением.

Так же через библиотеку scipy можем проверить правильность нахождения L, U и P матриц LUP-разложения.

Однако существуют случаи, когда решение СЛАУ невозможно методом LU-разложения. 

Решим систему 
\begin{equation*}
 \begin{cases}
   x_1 + 2 x_3 = 0\\
   3 x_1 + 2 x_3 = 1\\
   x_1 + 2 x_2 + 4 x_3 = 0
 \end{cases}
\end{equation*}

Её аналитическое решение: 
\begin{equation*}
 \begin{cases}
   x_1 = \frac{1}{2}\\
   x_2 = \frac{1}{4}\\
   x_3 = -\frac{1}{4}
 \end{cases}
\end{equation*}

При попытке найти LU-разложение случилась бы ситуация, при которой пришлось бы делить на ноль. Поэтому найти решение таким способом невозможно. Но для LUP-разложения этого не возникает.

In [133]:
matrix = np.array([[1.,0,2],[3,0,2],[1,2,4]])
vector = np.array([0.,1,0])

In [134]:
solve_LU(matrix,vector)

'Главный минор вырожден'

In [135]:
L, U, P = LUP(matrix)
x = solve_LUP(matrix,vector)
print('L =', L)
print('U =', U)
print('P =', P)
for i in range(len(matrix)):
    print(f'x_{i + 1} = {x[i]}')

L = [[1.         0.         0.        ]
 [0.33333333 1.         0.        ]
 [0.33333333 0.         1.        ]]
U = [[3.         0.         2.        ]
 [0.         2.         3.33333333]
 [0.         0.         1.33333333]]
P = [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
x_1 = 0.5
x_2 = 0.24999999999999997
x_3 = -0.24999999999999997


In [136]:
sl.solve(matrix,vector)
P, L, U = sl.lu(matrix)
print('L =', L)
print('U =', U)
print('P =', P.T)
for i in range(len(matrix)):
    print(f'x_{i + 1} = {x[i]}')

L = [[1.         0.         0.        ]
 [0.33333333 1.         0.        ]
 [0.33333333 0.         1.        ]]
U = [[3.         0.         2.        ]
 [0.         2.         3.33333333]
 [0.         0.         1.33333333]]
P = [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
x_1 = 0.5
x_2 = 0.24999999999999997
x_3 = -0.24999999999999997
